### Approach 1. Import Transformer Models

In [3]:
from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline

In [12]:
from transformers import BertTokenizer, BertForTokenClassification, pipeline

# Load the pre-trained model and tokenizer
model_name = "cl-tohoku/bert-base-japanese"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name)

# Create a NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Example text
text = "東京都にある株式会社サンプルの田中太郎です。"
ner_results = nlp(text)

# Print the results
for entity in ner_results:
    print(f"Entity: {entity['word']}, Label: {entity['entity']}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Entity: 東, Label: LABEL_0
Entity: 京, Label: LABEL_0
Entity: 都, Label: LABEL_0
Entity: にあ, Label: LABEL_1
Entity: ##る, Label: LABEL_0
Entity: 株, Label: LABEL_0
Entity: 式, Label: LABEL_1
Entity: 会, Label: LABEL_0
Entity: 社, Label: LABEL_0
Entity: サンプル, Label: LABEL_0
Entity: ##の, Label: LABEL_0
Entity: 田, Label: LABEL_0
Entity: 中, Label: LABEL_0
Entity: 太, Label: LABEL_0
Entity: 郎, Label: LABEL_0
Entity: です, Label: LABEL_0
Entity: 。, Label: LABEL_1


### Approach 2. LLMs

In [1]:
from dotenv import load_dotenv
import os

In [2]:

japText = "「私の名前は小林直樹です。」"

In [3]:
import openai
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# Load environment variables from a .env file
load_dotenv()

# Set up OpenAI API key
#client = OpenAI(api_key=os.getenv('O_API_KEY'))
api_key=os.getenv('O_API_KEY')



In [4]:
japListOfPII = ['「私の名前は田中太郎です。」',
 '「名前は山田花子です。」',
 '「私は鈴木健太です。」',
 '「名前は佐藤みゆきです。」',
 '「私の名前は小林直樹です。」',
 '「私は伊藤美香です。」',
 '「名前は高田健太郎です。」',
 '「私の名前は中村かおりです。」',
 '「名前は大野隆夫です。」',
 '「私は田辺さちこです。」',
 '「生年月日は1990年3月15日です。」',
 '「誕生日は1985年10月5日です。」',
 '「生年月日は1978年6月20日です。」',
 '「誕生日は1995年2月8日です。」',
 '「生年月日は1983年9月12日です。」',
 '「誕生日は2000年7月25日です。」',
 '「生年月日は1970年1月1日です。」',
 '「誕生日は1992年11月18日です。」',
 '「生年月日は1987年4月30日です。」',
 '「誕生日は2005年12月10日です。」',
 '「保険証券番号はABC123456です。」',
 '「保険証番号はXYZ789012です。」',
 '「保険の契約番号はDEF456789です。」',
 '「保険証書の番号はLMN654321です。」',
 '「保険契約書の番号はPQR987654です。」',
 '「保険ポリシー番号はGHI345678です。」',
 '「契約のポリシー番号はJKL876543です。」',
 '「保険の証明書番号はSTU234567です。」',
 '「契約書の保険番号はNOP543210です。」',
 '「保険の契約証券番号はQRS765432です。」',
 '「現在の住所は東京都渋谷区青山1-1-1です。」',
 '「住所は大阪市中央区難波2-3-4です。」',
 '「現住所は神奈川県横浜市中区山手町5-6-7です。」',
 '「住所は北海道札幌市中央区北1条西2-3です。」',
 '「現在の住所は福岡県福岡市博多区博多駅前1-2-3です。」',
 '「住所は京都府京都市中京区四条通り1-1です。」',
 '「現住所は愛知県名古屋市中区栄3-4-5です。」',
 '「住所は広島県広島市中区本通り6-7です。」',
 '「現在の住所は沖縄県那覇市おもろまち1-2-3です。」',
 '「住所は岐阜県岐阜市金町2-3-4です。」']

In [5]:
def extractFronJapText(japText): 
    prompt_and_context = [
        #("human", "{query}."),
        ("system", """You are an expert data entry operator in Japanese. Extract PII data such as name , date , birth or policy number , address 
        from the given sentence. You must only return response in a python dictionary with key valye pair - like name:extractedName,date_of_birth:extracted_date_of_birth. If there is no PII information return null in the key:value format. #### Context:{context}. These are PII information accuracy is very important think step by step"""),
    ]
    chat = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key, temperature=0.1)
    chat_template = ChatPromptTemplate.from_messages(prompt_and_context)
    message = chat_template.format_messages(context=japText)
    ai_resp = chat.invoke(message)
    return ai_resp.content

In [17]:
def extractFronJapText(japText): 
    system_message = """You are an expert data entry operator in Japanese. 
                        Extract PII data such as name, date of birth, policy number, 
                        and address from the given sentence. You must only return the 
                        extracted PII in a Python dictionary with key-value pairs - 
                        like name: extractedName, date_of_birth: extracted_date_of_birth 
                        else return null. The response must be in Japanese. 
                        
                        #### 
                        Context: {context}.
                        These are PII information accuracy is very important think step by step."""
    
    prompt_and_context = [
        ("system",system_message),
        ("user", japText)
    ]
    chat = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key, temperature=0.1)
    chat_template = ChatPromptTemplate.from_messages(prompt_and_context)
    message = chat_template.format_messages(context=japText)
    ai_resp = chat.invoke(message)
    return ai_resp.content

In [18]:
t1 = "「保険ポリシー番号はGHI345678です。」"

In [19]:
extractFronJapText(t1)

'{\n    "policy_number": "GHI345678"\n}'

In [49]:
def extract_PII_Japanese_Text(japText): 
    prompt_and_context = [
        #("human", "{query}."),
        ("system", """You are an expert data entry operator in Japanese. Extract PII data such as name or date or birth or policy number or address 
        from the given sentence. You must only return response in a python dictionary with key valye pair - like name:extractedName,date_of_birth:extracted_date_of_birth. If there is no PII information return null only retunr key that has a value - dont make mistake. #### Context:{context}. These are PII information accuracy is very important think step by step"""),
    ]
    chat = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key, temperature=0.1)
    chat_template = ChatPromptTemplate.from_messages(prompt_and_context)
    message = chat_template.format_messages(context=japText)
    ai_resp = chat.invoke(message)
    return ai_resp.content

In [47]:
%%time
for items in japListOfPII:
    extVale = extract_PII_Japanese_Text(items)
    print(f"OriginalText: {items}, extractedText: {extVale}")

OriginalText: 「私の名前は田中太郎です。」, extractedText: {
    "name": "田中太郎"
}
OriginalText: 「名前は山田花子です。」, extractedText: {
    "name": "山田花子"
}
OriginalText: 「私は鈴木健太です。」, extractedText: {
    "name": "鈴木健太"
}
OriginalText: 「名前は佐藤みゆきです。」, extractedText: {
    "name": "佐藤みゆき",
    "date_of_birth": null,
    "policy_number": null,
    "address": null
}
OriginalText: 「私の名前は小林直樹です。」, extractedText: {
    "name": "小林直樹"
}
OriginalText: 「私は伊藤美香です。」, extractedText: {
    "name": "伊藤美香"
}
OriginalText: 「名前は高田健太郎です。」, extractedText: {
    "name": "高田健太郎"
}
OriginalText: 「私の名前は中村かおりです。」, extractedText: {
    "name": "中村かおり"
}
OriginalText: 「名前は大野隆夫です。」, extractedText: {
    "name": "大野隆夫"
}
OriginalText: 「私は田辺さちこです。」, extractedText: {
    "name": "田辺さちこ"
}
OriginalText: 「生年月日は1990年3月15日です。」, extractedText: {
    "date_of_birth": "1990年3月15日"
}
OriginalText: 「誕生日は1985年10月5日です。」, extractedText: {
    "date_of_birth": "1985年10月5日"
}
OriginalText: 「生年月日は1978年6月20日です。」, extractedText: {
    "date_of_birth": "19

In [48]:



# Example text
text = "東京都にある株式会社サンプルの田中太郎です。"

# Extract entities using OpenAI
entities = openai_extract_entities(text)
print(entities)



NameError: name 'openai_extract_entities' is not defined

In [24]:
import logging
logging.basicConfig(level=logging.INFO)
try:
    # Your code that may raise an exception
    raise ValueError("An example error occurred")
except Exception as e:
    # Log the exception
    logging.error(f"Exception occurred: {str(e)}")

ERROR:root:Exception occurred: An example error occurred


### STT

In [40]:
with open("audio/test.webm", "rb") as f:
    audio_content = f.read()

In [43]:
client = OpenAI(api_key=api_key)
# Call OpenAI Whisper model
transcription = client.audio.transcriptions.create(model="whisper-1",language='en',file = open("audio/utterance.webm", "rb"),response_format="text")
transcription

'This is mic testing, mic testing, hello, mic testing, hello, hello, mic testing.\n'

In [42]:
transcription

'Mic testing, mic testing, yellow mic testing.\n'